ETL

In [ ]:
import pandas as pd
import json
import ast

In [48]:
with open('output_steam_games.json', 'r') as file:
  lines = file.readlines()

steamGamesData = [json.loads(line) for line in lines]

steamGames = pd.json_normalize(steamGamesData)

steamGamesDf=pd.DataFrame(steamGames)

In [49]:
steamGamesDf['release_date']=pd.to_datetime(steamGamesDf['release_date'],format='mixed',errors='coerce')
steamGamesDf['Año'] = steamGamesDf['release_date'].dt.year.fillna(0).astype(int)

In [50]:
steamGamesDf

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,Año
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",2018
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,2018
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,2018
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",2017


In [ ]:
steamGamesDf.to_json('steamGames.json')
steamGamesDf.to_csv('steamGames.csv')

In [ ]:
steamGamesPriceDf=steamGamesDf[['id','price']]
steamGamesPriceDf.to_json('steamGamesPrice.json')
steamGamesPriceDf.to_csv('steamGamesPrice.csv')

In [ ]:
#Ruta del archivo JSON
data_list = []
file_path = 'australian_users_items.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
userItemsDf = pd.DataFrame(data_list)

In [ ]:
userItemsDf.to_json('userItems.json')
userItemsDf.to_csv('userItems.csv')

In [43]:
userItemsExplodedDf=userItemsDf.explode('items').reset_index(drop=True)

In [44]:
userItemsExplodedDf['items']=userItemsExplodedDf['items'].apply(lambda x: x.get('item_id') if isinstance(x, dict) and 'item_id' in x else None)
userItemsExplodedDf.rename(columns={'items': 'item_id'}, inplace=True)

In [45]:
del userItemsExplodedDf['user_url']

In [46]:
del userItemsExplodedDf['items_count']

Def Developer

In [56]:
def developer(dev):
  steamGamesData=pd.read_csv('steamGames.csv')
  steamGamesDf=pd.DataFrame(steamGamesData)
  filteredDf=steamGamesDf[steamGamesDf['developer']==dev]
  totalApps=filteredDf.groupby('Año').size().reset_index(name='Cantidad de Items')
  freeApps=filteredDf[filteredDf['price'].str.lower() == 'free'].groupby('Año').size().reset_index(name='free_items')
  result = pd.merge(totalApps, freeApps, on='Año', how='left').fillna(0)
  result['Contenido Free'] = ((result['free_items'] / result['Cantidad de Items']) * 100).round(2).astype(str) + '%'
  result=result[['Año','Cantidad de Items','Contenido Free']].reset_index(drop=True)
  return result

In [54]:
developer('AMPLITUDE Studios')

C:\Users\Admin\AppData\Local\Temp\ipykernel_5128\3308200209.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  steamGamesData=pd.read_csv('steamGames.csv')


,Año,Cantidad de Items,Contenido Free
0,2012,1,0.0%
1,2014,3,0.0%
2,2015,9,22.22%
3,2016,2,0.0%
4,2017,8,87.5%


Def Userdata

In [47]:
userItemsExplodedDf

,user_id,steam_id,item_id
0,76561197970982479,76561197970982479,10
1,76561197970982479,76561197970982479,20
2,76561197970982479,76561197970982479,30
3,76561197970982479,76561197970982479,40
4,76561197970982479,76561197970982479,50
...,...,...,...
5170010,76561198329548331,76561198329548331,373330
5170011,76561198329548331,76561198329548331,388490
5170012,76561198329548331,76561198329548331,521570
5170013,76561198329548331,76561198329548331,519140
